In [1]:
import os
import copy
import time
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from utils.config import Config
from nets.yolo3 import YoloBody
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from nets.yolo_training import YOLOLoss, Generator
from utils.dataloader import yolo_dataset_collate, YoloDataset

In [2]:
def fit_one_epoch(net, yolo_losses, epoch, epoch_size, epoch_size_val, gen, genval, Epoch, cuda, optimizer, lr_scheduler):
    total_loss = 0
    val_loss = 0
    print('\n' + '-' * 10 + 'Train one epoch.' + '-' * 10)
    print('Epoch:'+ str(epoch+1) + '/' + str(Epoch))
    print('Start Training.')
    net.train()
    for iteration, batch in enumerate(gen):
        start_time = time.time()
        if iteration >= epoch_size:
            break
        images, targets = batch[0], batch[1]
        with torch.no_grad():
            if cuda:
                images = Variable(torch.from_numpy(images).type(torch.FloatTensor)).cuda()
                targets = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets]
            else:
                images = Variable(torch.from_numpy(images).type(torch.FloatTensor))
                targets = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets]
        optimizer.zero_grad()
        outputs = net(images)
        losses = []
        for i in range(3):
            loss_item = yolo_losses[i](outputs[i], targets)
            losses.append(loss_item[0])
        loss = sum(losses)
        loss.backward()
        optimizer.step()

        total_loss += loss
        waste_time = time.time() - start_time
        if iteration == 0 or (iteration+1) % 10 == 0:
            print('step:' + str(iteration+1) + '/' + str(epoch_size) + ' || Total Loss: %.4f || %.4fs/step' % (total_loss/(iteration+1), waste_time))
    lr_scheduler.step()
    print('Finish Training.')

    '''
    print('Start Validation')
    net.eval()
    for iteration, batch in enumerate(genval):
        if iteration >= epoch_size_val:
            break
        images_val, targets_val = batch[0], batch[1]

        with torch.no_grad():
            if cuda:
                images_val = Variable(torch.from_numpy(images_val).type(torch.FloatTensor)).cuda()
                targets_val = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets_val]
            else:
                images_val = Variable(torch.from_numpy(images_val).type(torch.FloatTensor))
                targets_val = [Variable(torch.from_numpy(ann).type(torch.FloatTensor)) for ann in targets_val]
            optimizer.zero_grad()
            outputs = net(images_val)
            losses = []
            for i in range(3):
                loss_item = yolo_losses[i](outputs[i], targets_val)
                losses.append(loss_item[0])
            loss = sum(losses)
            val_loss += loss
    print('Finish Validation')
    '''
    
    print('Total Loss: %.4f || Val Loss: %.4f ' % (total_loss/(epoch_size+1), val_loss/(epoch_size_val+1)))

    return total_loss/(epoch_size+1), val_loss/(epoch_size_val+1)

In [3]:
# 参数初始化
train_annotation_path = 'model_data/voc_train.txt'
val_annotation_path = 'model_data/voc_val.txt'
model = YoloBody(Config)
Cuda = True
#-------------------------------#
#   Dataloder的使用
#-------------------------------#
Use_Data_Loader = False

print('Loading pretrained model weights.')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dict = model.state_dict()
pretrained_dict = torch.load("model_data/yolo_weights.pth", map_location=device)
pretrained_dict = {k: v for k, v in pretrained_dict.items() if np.shape(model_dict[k]) ==  np.shape(v)}
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)
print('Finished!')

if Cuda:
    net = torch.nn.DataParallel(model)
    cudnn.benchmark = True
    net = net.cuda()

# 建立loss函数
yolo_losses = []
for i in range(3):
    yolo_losses.append(YOLOLoss(np.reshape(Config["yolo"]["anchors"],[-1,2]),
                                Config["yolo"]["classes"], (Config["img_w"], Config["img_h"]), Cuda))

with open(train_annotation_path) as f:
    train_lines = f.readlines()
with open(val_annotation_path) as f:
    val_lines = f.readlines()
num_train = len(train_lines)
num_val = len(val_lines)

Loading pretrained model weights.
Finished!


In [4]:
#------------------------------------#
#   冻结一定部分训练
#------------------------------------#
lr = 1e-3
Batch_size = 32
Init_Epoch = 0
Freeze_Epoch = 25
        
optimizer = optim.Adam(net.parameters(), lr)
#optimizer = optim.SGD(net.parameters(), lr, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

if Use_Data_Loader:
    train_dataset = YoloDataset(train_lines, (Config["img_h"], Config["img_w"]))
    val_dataset = YoloDataset(val_lines, (Config["img_h"], Config["img_w"]))
    gen = DataLoader(train_dataset, batch_size=Batch_size, num_workers=8, pin_memory=True,
                            drop_last=True, collate_fn=yolo_dataset_collate)
    gen_val = DataLoader(val_dataset, batch_size=Batch_size, num_workers=8,pin_memory=True, 
                            drop_last=True, collate_fn=yolo_dataset_collate)
else:
    gen = Generator(Batch_size, train_lines,
                        (Config["img_h"], Config["img_w"])).generate()
    gen_val = Generator(Batch_size, val_lines,
                        (Config["img_h"], Config["img_w"])).generate()
                        
epoch_size = num_train//Batch_size
epoch_size_val = num_val//Batch_size
for param in model.backbone.parameters():
    param.requires_grad = False

best_loss = 99999999.0
best_model_weights = copy.deepcopy(model.state_dict())
for epoch in range(Init_Epoch, Freeze_Epoch):
    total_loss, val_loss = fit_one_epoch(net, yolo_losses, epoch, epoch_size, epoch_size_val, gen, gen_val, 
                                         Freeze_Epoch, Cuda, optimizer, lr_scheduler)
    if total_loss < best_loss:
        best_loss = total_loss
        best_model_weights = copy.deepcopy(model.state_dict())
    with open('total_loss.csv', mode='a+') as total_loss_file:
        total_loss_file.write(str(total_loss.item()) + '\n')
    #with open('val_loss.csv', mode='a+') as val_loss_file:
    #    val_loss_file.write(str(val_loss.item()) + '\n')
torch.save(best_model_weights, 'model_data/yolov3_voc_weights0.pth')


----------Train one epoch.----------
Epoch:1/25
Start Training.
step:1/156 || Total Loss: 7803.9331 || 2.0100s/step
step:10/156 || Total Loss: 3594.3267 || 0.4677s/step
step:20/156 || Total Loss: 2108.3865 || 0.5276s/step
step:30/156 || Total Loss: 1486.1564 || 0.4780s/step
step:40/156 || Total Loss: 1150.4677 || 0.4558s/step
step:50/156 || Total Loss: 941.8665 || 0.4864s/step
step:60/156 || Total Loss: 799.6990 || 0.4587s/step
step:70/156 || Total Loss: 697.1407 || 0.4572s/step
step:80/156 || Total Loss: 619.0457 || 0.4835s/step
step:90/156 || Total Loss: 557.6161 || 0.4859s/step
step:100/156 || Total Loss: 507.9505 || 0.4769s/step
step:110/156 || Total Loss: 466.9517 || 0.4825s/step
step:120/156 || Total Loss: 432.6122 || 0.4912s/step
step:130/156 || Total Loss: 403.4812 || 0.5004s/step
step:140/156 || Total Loss: 377.9802 || 0.4966s/step
step:150/156 || Total Loss: 355.8572 || 0.4681s/step
Finish Training.
Total Loss: 341.5372 || Val Loss: 0.0000 

----------Train one epoch.-------

In [ ]:
#------------------------------------#
#   解冻后训练
#------------------------------------#            
lr = 1e-4
Batch_size = 16
Freeze_Epoch = 25
Unfreeze_Epoch = 50

optimizer = optim.Adam(net.parameters(), lr)
#optimizer = optim.SGD(net.parameters(), lr, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)
if Use_Data_Loader:
    train_dataset = YoloDataset(train_lines, (Config["img_h"], Config["img_w"]))
    val_dataset = YoloDataset(val_lines, (Config["img_h"], Config["img_w"]))
    gen = DataLoader(train_dataset, batch_size=Batch_size, num_workers=8, pin_memory=True,
                            drop_last=True, collate_fn=yolo_dataset_collate)
    gen_val = DataLoader(val_dataset, batch_size=Batch_size, num_workers=8,pin_memory=True, 
                            drop_last=True, collate_fn=yolo_dataset_collate)
else:
    gen = Generator(Batch_size, train_lines,
                        (Config["img_h"], Config["img_w"])).generate()
    gen_val = Generator(Batch_size, val_lines,
                        (Config["img_h"], Config["img_w"])).generate()
                        
epoch_size = num_train//Batch_size
epoch_size_val = num_val//Batch_size
for param in model.backbone.parameters():
    param.requires_grad = True

for epoch in range(Freeze_Epoch, Unfreeze_Epoch):
    total_loss, val_loss = fit_one_epoch(net, yolo_losses, epoch, epoch_size, epoch_size_val, gen, gen_val, 
                                         Unfreeze_Epoch, Cuda, optimizer, lr_scheduler)
    if total_loss < best_loss:
        best_loss = total_loss
        best_model_weights = copy.deepcopy(model.state_dict())
    with open('total_loss.csv', mode='a+') as total_loss_file:
        total_loss_file.write(str(total_loss.item()) + '\n')
    #with open('val_loss.csv', mode='a+') as val_loss_file:
    #    val_loss_file.write(str(val_loss.item()) + '\n')
torch.save(best_model_weights, 'model_data/yolov3_voc_weights1.pth')